European Central Bank (ECB) API with Python 2.7
=====

## Data on Eurozone countries

-----

*September 3, 2017*<br>
*@bd_econ*

ECB API Documentation is [here](https://sdw-wsrest.ecb.europa.eu/web/generator/index.html)

This example requests the interest rate by country on 10-year bonds over the period since 2012.

In [1]:
import requests
import pandas as pd

## Parameters/Settings

List of data sets is [here](https://sdw-wsrest.ecb.europa.eu/service/dataflow)

In [2]:
# List of countries
c_list = ['DE', 
          'FR', 
          'IT', 
          'ES', 
          'PT', 
          'GR', 
          'NL', 
          'BE', 
          'AT', 
          'IE']

param = [('dataflow', 'IRS'),
         ('freq', 'M'),
         ('countries', '+'.join(c_list)),
         ('series', 'L'),
         ('trans type', 'L40'),
         ('maturity cat', 'CI'),
         ('counterpart', '0000'),
         ('currency', 'EUR'),
         ('business coverage', 'N'),
         ('interest rate type', 'Z'),
         ('start', '?startPeriod=2012-01-01')]

param_joined = '.'.join(value for key, value in param[1:-1])

series_key = '{}/{}{}'.format(param[0][1],
                              param_joined,
                              param[-1][1])

## Request data

In [3]:
url = 'https://sdw-wsrest.ecb.europa.eu/service/data/'
# headers used as content negotiation to return data in json format
headers = {'Accept':'application/json'}
r = requests.get('{}{}'.format(url, series_key), headers=headers).json()

## Pandas DataFrame

In [5]:
date_list = r['structure']['dimensions']['observation'][0]['values']
dates = [date['start'][:10] for date in date_list]
    
areas = [v['name'] for v in r['structure']['dimensions']['series'][1]['values']]

df = pd.DataFrame()
for i, area in enumerate(areas):
    s_key = '0:{}:0:0:0:0:0:0:0'.format(i)
    s_list = r['dataSets'][0]['series'][s_key]['observations']
    df[area] = pd.Series([s_list[val][0] for val in sorted(s_list, key=int)])
df.index = dates
df.tail()

,Austria,Belgium,Germany,Spain,France,Greece,Ireland,Italy,Netherlands,Portugal
2017-03-01,0.5924,0.87,0.35,1.717,1.02,7.17,1.05,2.401,0.485,3.992
2017-04-01,0.4903,0.78,0.22,1.613,0.88,6.70,0.91,2.257,0.498,3.774
2017-05-01,0.6517,0.77,0.34,1.571,0.81,5.86,0.83,2.194,0.589,3.287
2017-06-01,0.5547,0.62,0.25,1.453,0.66,5.76,0.70,2.053,0.503,2.968
2017-07-01,0.7292,0.83,0.46,1.599,0.84,5.33,0.87,2.231,0.688,3.025
